In [32]:
#Imports

#Data
import pandas as pd
import numpy as np

#Date
import datetime as dt
from datetime import datetime

#Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#ML
from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler



from sklearn.neighbors import KNeighborsClassifier

from sklearn import datasets
from sklearn.metrics import confusion_matrix,classification_report

from sklearn.preprocessing import OneHotEncoder

data_file='wrang_xyz_data.csv'

financial_data=['total_rev_hi_lim','tot_cur_bal','tot_coll_amt','revol_util','loan_amnt','dti','revol_bal']
loan_data=['purpose','initial_list_status','term']
emp_data=['emp_length','collections_12_mths_ex_med','acc_now_delinq','home_ownership','annual_inc','verification_status','zip_code','addr_state','delinq_2yrs','inq_last_6mths','open_acc','pub_rec','total_acc']
date_data=['earliest_cr_line']


#out=['last_pymnt_d','last_credit_pull_d','initial_list_status','recoveries','collection_recovery_fee','last_pymnt_amnt','total_pymnt','total_rec_int','int_rate','out_prncp',''total_rec_late_fee','default_ind']

features=financial_data+loan_data+emp_data+date_data
pred_data=['issue_d','grade','sub_grade']

In [40]:
def get_model_class(data_file='wrang_xyz_data.csv',split='random',pred_value=['grade']):
 
    df = pd.read_csv('data/'+data_file,usecols=features+['issue_d']+pred_value)




    if split=='random':
        X=df.drop([pred_value[0],'issue_d'],axis=1)
        y=df[pred_value[0]]

        scaler = StandardScaler()
        scaler.fit(X)
        X = scaler.transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

    else:
        X=df.drop([pred_value[0]],axis=1)
        y=df[[pred_value[0],'issue_d']]


        X_train=X[X['issue_d']<=20150501].drop('issue_d',axis=1)
        X_test=X[X['issue_d']>20150501].drop('issue_d',axis=1)

        y_train=y[y['issue_d']<=20150501][pred_value[0]]
        y_test=y[y['issue_d']>20150501][pred_value[0]]

        scaler = StandardScaler()
        scaler.fit(X_train,X_test)
        X_train = scaler.transform(X_train)


    model_knn = KNeighborsClassifier(n_neighbors=5)
    model_knn.fit(X_train,y_train)
    y_knn_pred = knn.predict(X_test)
    print(confusion_matrix(y_test, y_knn_pred))
    print(classification_report(y_test, y_knn_pred))

    return model_knn

model_knn=get_model_class(split='random',pred_value=['grade'])

c:\Users\mmoy\Documents\XYZ Corp\xyz_corp_venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


KeyboardInterrupt: 